In [1]:
import re
import pandas as pd
df = pd.read_excel("./work.xlsx")

In [2]:
# Main data (extracted from Excel)
cty_codes = pd.read_csv("./ICS_CountryCodes.csv")
df_merged = pd.merge(df, cty_codes, left_on='cty_code', right_on='ISO3', how='left')
iso3_to_name = dict(zip(cty_codes['ISO3'], cty_codes['CtyName']))
iso3_to_std = dict(zip(cty_codes['ISO3'], cty_codes['Standard']))
df['cty_name'] = df['cty_code'].apply(lambda x: iso3_to_name.get(x, ''))
df['standard'] = df['cty_code'].apply(lambda x: iso3_to_std.get(x, ''))
df_work = df

In [3]:
# Group main data by country + year
df_work['Avg'] = df[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dec']].apply(lambda row: row[row.apply(lambda x: isinstance(x, (int, float)))].mean(), axis=1)
df_grouped = df_work[df_work['P_or_T'] == 'T'].groupby(['cty_name', 'cty_code', 'year', 'standard']).agg({
    'Annual': 'sum',
    'Quarterly': 'sum',
    'Monthly': 'sum',
    'Total': 'sum',
    'Avg': 'mean'
}).reset_index()
df_nsdp = df_grouped

In [4]:
# HDR indices from https://hdr.undp.org/data-center/documentation-and-downloads
_df = pd.read_csv('HDR21-22_Composite_indices_complete_time_series.csv')
df_hdi_set = {}
for c in ['hdi', 'le', 'eys', 'mys', 'gnipc']:
    columns = [col for col in _df.columns if re.match(c + '_[0-9]{4}', col)]
    df_reshaped = pd.melt(_df, id_vars=['iso3'], value_vars=columns, var_name='year', value_name=c)
    df_reshaped['yr'] = df_reshaped['year'].apply(lambda x: x[-4:])
    df_le = df_reshaped.drop(['year'], axis=1)
    df_le.rename(columns={'yr': 'year'}, inplace=True)
    df_le.dropna(subset=[c], inplace=True)
    df_hdi_set[c] = df_le

In [5]:
# CPI index from https://www.transparency.org/en/
_df = pd.read_excel('./CPI2022_GlobalResultsTrends.xlsx', sheet_name='CPI Timeseries 2012 - 2022', skiprows=2) 
hdi_columns = [col for col in _df.columns if re.match(r'CPI score [0-9]{4}', col, re.IGNORECASE) or re.match(r'Standard error [0-9]{4}', col)]
df_reshaped = pd.melt(_df, id_vars=['ISO3'], value_vars=hdi_columns, var_name='label', value_name='value')
df_reshaped['year'] = df_reshaped['label'].apply(lambda x: x[-4:])
df_reshaped['type'] = df_reshaped['label'].apply(lambda x: 'cpi' if x.startswith('CPI') else 'se_cpi')
df_cpi = df_reshaped.pivot(index=['ISO3', 'year'], columns='type', values='value').reset_index()

In [6]:
# Freedom score from https://www.heritage.org/index/about
_df = pd.read_excel('./freedom-scores.xlsx')
_df.rename(columns={'Overall Score': 'ef_score', 'Index Year': 'year'}, inplace=True)
df_fs = _df.drop(columns=set(_df.columns) - set(['ISO3', 'year', 'ef_score'])).dropna(subset=['ef_score'])
duplicates_fs = df_fs[df_fs.duplicated(subset=['ISO3', 'year'], keep=False)]
if duplicates_fs.empty:
    print("No duplicates found in df_fs.")
else:
    print("Duplicates found in df_fs:")
    print(duplicates_fs)

No duplicates found in df_fs.


In [7]:
# Gini index from https://www.wider.unu.edu/node/236613
_df = pd.read_excel('./WIID_06MAY2020.xlsx')
_df.rename(columns={'c3': 'ISO3', 'gini_reported': 'gini'}, inplace=True)
df_gini = _df.drop(columns=set(_df.columns) - set(['ISO3', 'year', 'gini'])).dropna(subset=['gini'])
df_gini.drop_duplicates(subset=['ISO3', 'year'], keep='last', inplace=True)

In [8]:
# WEO from https://www.imf.org/en/Publications/WEO/weo-database/2023/April
_df = pd.read_excel('WEOApr2023all.xlsx')
columns = [col for col in _df.columns if re.match(r'^[0-9]+$', str(col)) and int(col) >= 2009 and int(col) < 2023]
df_weo_set = {}
for c in ['NGDPDPC', 'LP', 'GGR', 'GGXWDG', 'GGX', 'NGSD_NGDP', 'NID_NGDP']:
    df_reshaped = pd.melt(_df[_df['WEO Subject Code'] == c], id_vars=['ISO'], value_vars=columns, var_name='year', value_name=c)
    df_reshaped.dropna(subset=[c], inplace=True)
    df_weo_set[c] = df_reshaped

In [9]:
# WGI from https://info.worldbank.org/governance/wgi/
info = {'VoiceandAccountability': 'voice', 'Political StabilityNoViolence': 'polstab', 'GovernmentEffectiveness': 'goveff', 'RegulatoryQuality': 'regqual', 'RuleofLaw': 'rulelaw', 'ControlofCorruption': 'contrcorr'}
df_wgi_set = {}
for k,v in info.items():
    _df = pd.read_excel('wgidataset.xlsx', sheet_name=k, skiprows=range(0,14))
    names = ["Estimate"] + ["Estimate.{0}".format(i) for i in range(1,23)]
    yrs = [1996, 1998, 2000, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    columns = {i[0]:i[1] for i in zip(names, yrs)}
    columns['Code'] = 'iso3'
    _df.rename(columns=columns, inplace=True)
    _df_reshaped = pd.melt(_df, id_vars=['iso3'], value_vars=yrs, var_name='year', value_name=v)
    _df_reshaped.dropna(subset=[v], inplace=True)
    df_wgi_set[v] = _df_reshaped

In [10]:
# Polity score from https://www.systemicpeace.org/polityproject.html
_df = pd.read_excel('./p5v2018.xls')
_df.rename(columns={'scode': 'ISO3'}, inplace=True)
df_polity = _df.drop(columns=set(_df.columns) - set(['ISO3', 'year', 'polity']))

In [11]:
# Merging datasets
# Set all years to type STR (avoid conflicts)
df_nsdp['year'] = df_nsdp['year'].astype(str)
df_gini['year'] = df_gini['year'].astype(str)
df_cpi['year'] = df_cpi['year'].astype(str)
df_fs['year'] = df_fs['year'].astype(str)
df_polity['year'] = df_polity['year'].astype(str)
for c in ['hdi', 'le', 'eys', 'mys', 'gnipc']:
    df_hdi_set[c]['year'] = df_hdi_set[c]['year'].astype(str)
for c in ['NGDPDPC', 'LP', 'GGR', 'GGXWDG', 'GGX',  'NGSD_NGDP', 'NID_NGDP']:
    df_weo_set[c]['year'] = df_weo_set[c]['year'].astype(str)
for c in ['voice', 'polstab', 'goveff', 'regqual', 'rulelaw', 'contrcorr']:
    df_wgi_set[c]['year'] = df_wgi_set[c]['year'].astype(str)

# Filter to useful years (there is no NSDP info before 2009)
df_gini_b = df_gini[df_gini['year'] >= '2009']
df_cpi_b = df_cpi[df_cpi['year'] >= '2009']
df_fs_b = df_fs[df_fs['year'] >= '2009']
df_polity_b = df_polity[df_polity['year'] >= '2009']

# Merge 
merged_df = df_nsdp.merge(df_gini_b, left_on=['cty_code', 'year'], right_on=['ISO3', 'year'], how='outer')
merged_df = merged_df.merge(df_cpi_b, left_on=['cty_code', 'year'], right_on=['ISO3', 'year'], how='outer')
merged_df = merged_df.merge(df_fs_b, left_on=['cty_code', 'year'], right_on=['ISO3', 'year'], how='outer')
merged_df = merged_df.merge(df_polity_b, left_on=['cty_code', 'year'], right_on=['ISO3', 'year'], how='outer', suffixes=(c + '_x', c + 'yield'))

for c in ['hdi', 'le', 'eys', 'mys', 'gnipc']:
    merged_df = merged_df.merge(df_hdi_set[c], left_on=['cty_code', 'year'], right_on=['iso3', 'year'], how='outer', suffixes=(c + '_x', c + 'yield'))

for c in ['NGDPDPC', 'LP', 'GGR', 'GGXWDG', 'GGX', 'NGSD_NGDP', 'NID_NGDP']:
    merged_df = merged_df.merge(df_weo_set[c], left_on=['cty_code', 'year'], right_on=['ISO', 'year'], how='outer', suffixes=(c + '_x', c + 'yield'))
    
for c in ['voice', 'polstab', 'goveff', 'regqual', 'rulelaw', 'contrcorr']:
    merged_df = merged_df.merge(df_wgi_set[c], left_on=['cty_code', 'year'], right_on=['iso3', 'year'], how='outer', suffixes=(c + '_x', c + 'yield'))

# Drop redudant columns with keys
merged_df.drop(['cty_code', 'ISO'] + [c for c in merged_df.columns if re.match(r'iso3.+', c, re.IGNORECASE)] + [c for c in merged_df.columns if re.match(r'ISO.+', c)], axis=1, inplace=True)

# Remove null rows
_df = merged_df[~merged_df['cty_name'].isna() & (merged_df['cty_name'] != '') & (~merged_df['iso3'].isna())].copy()

# Flip signal of lags (original data is negative), it makes regression easier to interpret
_df['annual_neg'] = _df['Annual'].apply(lambda x: 0 - x)
_df['monthly_neg'] = _df['Monthly'].apply(lambda x: 0 - x)
_df['quarterly_neg'] = _df['Quarterly'].apply(lambda x: 0 - x)
_df['total_neg'] = _df['Total'].apply(lambda x: 0 - x)
_df.drop(['Annual', 'Monthly', 'Quarterly', 'Total', 'Avg'], axis=1, inplace=True)
_df.rename({'annual_neg': 'Annual', 'monthly_neg': 'Monthly', 'quarterly_neg': 'Quarterly', 'total_neg': 'Total'}, axis=1, inplace=True)

# Save and display
_df[['iso3', 'cty_name', 'year', 'standard', 'Annual', 'Quarterly', 'Monthly',
       'Total', 'cpi', 'ef_score', 'hdi', 'NGDPDPC', 'voice', 'polstab', 'goveff', 'regqual', 'rulelaw', 'polity']].to_excel('work-merged.xlsx')
display(_df[['iso3', 'cty_name', 'year', 'standard', 'Annual', 'Quarterly', 'Monthly',
       'Total', 'cpi', 'ef_score', 'hdi', 'NGDPDPC', 'voice', 'polstab', 'goveff', 'regqual', 'rulelaw', 'polity']])

,iso3,cty_name,year,standard,Annual,Quarterly,Monthly,Total,cpi,ef_score,hdi,NGDPDPC,voice,polstab,goveff,regqual,rulelaw,polity
5,ARG,Argentina,2013,SDDS,99.0,532.0,341.0,972.0,34.0,46.7,0.845,14488.829,0.276988,0.065314,-0.253653,-0.928135,-0.656233,8.0
6,ARG,Argentina,2015,SDDS,0.0,247.0,461.0,708.0,32.0,44.1,0.848,14895.316,0.411802,0.014785,-0.016539,-0.876747,-0.731064,9.0
7,ARG,Argentina,2016,SDDS,0.0,0.0,162.0,162.0,36.0,43.8,0.847,12772.868,0.493365,0.204626,0.250169,-0.337519,-0.304282,9.0
8,ARG,Argentina,2017,SDDS,0.0,0.0,12.0,12.0,39.0,50.4,0.851,14618.327,0.528664,0.169504,0.164413,-0.181595,-0.219417,9.0
9,ARG,Argentina,2018,SDDS,0.0,0.0,0.0,0.0,40.0,52.3,0.850,11786.433,0.530451,0.010891,0.054248,-0.183626,-0.200491,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,URY,Uruguay,2017,SDDS,0.0,20.0,25.0,45.0,70.0,69.7,0.819,18405.742,1.167405,1.050780,0.428820,0.655360,0.581297,NaN
632,URY,Uruguay,2018,SDDS,0.0,22.0,72.0,94.0,70.0,69.2,0.819,18408.701,1.180035,1.034957,0.530200,0.632341,0.595442,NaN
633,URY,Uruguay,2019,SDDS,0.0,0.0,52.0,52.0,71.0,68.6,0.821,17404.371,1.226977,1.038879,0.671302,0.622146,0.629673,NaN
634,URY,Uruguay,2020,SDDS,0.0,0.0,129.0,129.0,71.0,69.1,0.821,15168.311,1.311339,1.052871,0.769235,0.697773,0.681006,NaN


Regression was run in R, using the following:

```
library(psych)
library(readr)
library(readxl)
library(stargazer)
library(dplyr)
library(knitr)
library(kableExtra)

% load
work_merged <- read_excel("work-merged.xlsx")
work_merged$democracy <- ifelse(work_merged$polity > 0, 1, 0)

% regress
reg_model_T <- lm(Total ~ hdi + cpi + ef_score + NGDPDPC + voice + polstab + goveff + regqual + rulelaw + polity, data = work_merged)
reg_model_D <- lm(Annual ~ hdi + cpi + ef_score + NGDPDPC + voice + polstab + goveff + regqual + rulelaw + democracy, data = work_merged)
% print statistics
stargazer(as.data.frame(work_merged))
% print regression
stargazer(reg_model_T, reg_model_D,  title="Regression Results - Total delays per year", align=TRUE, single.row=TRUE, type="latex", no.space = TRUE, column.sep.width = "-20pt", font.size = "small")
```